<a href="https://colab.research.google.com/github/wilzamguerrero/SDZ/blob/main/SDZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Buy Me a cofee](https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/coffe4.gif?raw=true ) ](https://www.buymeacoffee.com/wilzamguerrero)

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/drive.png?raw=true' height="50" alt="netdata"/></center>
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()
print('[1;32mDrive Connected ✓')



#markdown Download tool
#import os
#from IPython.display import clear_output

#save_models_dir = "/content/sample_data"#@param {type:"string"}
#os.chdir(save_models_dir)
#url = "https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt" #@param {type:"string"}
#!wget --no-check-certificate $url

#clear_output()
#print('[1;32mModel Save ✓')
     

In [ ]:
#@markdown <left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/SDZanim2.gif?raw=true' height="50" alt="netdata"/></center> <a href="https://wilzamguerrero.ml/3c030fffd57c4e3bb4dd62e3971cb6a9"><font face='Verdana Regular' color='#80ff3' size='4'><left><img src='https://github.com/wilzamguerrero/SDZ/blob/main/SDZ_custom/icon/how_to_use5.png?raw=true' height="50" alt="netdata"/></center></font></a> 
##@markdown <left><font face='Squired Regular' color='#00f3f3' size='4'>Stable Diffusion Z</font>
##@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>SDZboton</font> <a href="https://pbs.twimg.com/profile_images/1022127431447195649/ASyO_yY3_400x400.jpg"><font face='Verdana Regular' color='#80ff3' size='4'>SDZboton</font></a>
import os
import shutil
from IPython.utils import capture
from IPython.display import clear_output
import sys
import fileinput
import time

#@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>▣</font>
SDZ_install = "/content" #@param {type:"string"} 
SDZ_Version = "V1.5" #@param ["V1.5", "V2.1"]
SDZ_Download_Model = "" #@param ["", "V1.5", "V1.5.B", "V1.5.inpaint", "V1.5.pix2pix", "V2.1"]
SDZ = "stable-diffusion-webui"
SDZ_webui_path = (SDZ_install+'/'+'stable-diffusion-webui')
os.chdir(SDZ_install)
#@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>▥</font>
SDZ_Model_Init = "" #@param {type:"string"}
SDZ_Models_Dir = "" #@param {type:"string"}
SDZ_Training = "" #@param ["", "New_Dreambooth - Kohya", "Old_Dreambooth"]
SDZ_Args = "" # @param {type:"string"}
SDZ_Extensions = False #@param {type:"boolean"}
SDZ_ControlNet = False #@param {type:"boolean"}
SDZ_Big_Models = True
SDZ_Upscalers = False #@param {type:"boolean"}
#@markdown <left><font face='Verdana Regular' color='#00f3f3' size='4'>◨</font>
SDZ_Continue = False #@param {type:"boolean"}
SDZconfigV2="--config v2-inference-v.yaml"
share = "--share --api --cors-allow-origins=https://www.painthua.com --xformers --theme=dark --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae" + " --ckpt=" + SDZ_Model_Init + " --ckpt-dir=" + SDZ_Models_Dir + " " + SDZ_Args

def getLink(path):
  with open(path, 'r') as testwritefile:
    webui_link = testwritefile.read()
    webui_link = webui_link[webui_link.rfind("http"):][:-1]
  return webui_link

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

# SDZ_RunSDZ
if not SDZ_Continue: 
  with capture.capture_output() as cap:
    if os.path.exists(SDZ):
      shutil.rmtree(SDZ)
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

    if not os.path.exists(SDZ):
      !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

    %cd stable-diffusion-webui
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user2.css -O user.css
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/Upscalers.txt
    
    clear_output()
  print('[1;96mInstalling SDZ ✓')

if SDZ_Continue:
  with capture.capture_output() as cap:
    os.chdir(SDZ_webui_path)
    clear_output()
  print('[1;96mPreparing SDZ Continue ✓')

# SDZ_Requirements
with capture.capture_output() as cap:
  os.chdir(SDZ_webui_path)
  import os
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
  !pip install triton
  #!pip install triton==2.0.0.dev20221202.
  #!git clone https://github.com/bytedance/Next-ViT.git externals/Next_ViT
  !wget -N https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

  #%cd models/Stable-diffusion
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/open_clip'):
    #!pip install xformers==0.0.16rc425
    !pip install open-clip-torch

# SDZ_Models
  os.chdir(SDZ_webui_path)
  %cd models/Stable-diffusion
  if SDZ_Download_Model == "V1.5":
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Zv1-5-fp16.ckpt -O v15.ckpt

  elif SDZ_Download_Model == "V1.5.B":
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-pruned-emaonly.safetensors -O v15B.safetensors

  elif SDZ_Download_Model == "V1.5.inpaint":
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/v1-5-inpaint-fp16.ckpt -O v15inp.ckpt

  elif SDZ_Download_Model == "V1.5.pix2pix":
    !wget --timestamping https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.safetensors -O v15pix2pix.safetensors

  elif SDZ_Download_Model == "V2.1":
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Zv2-1_768-fp16.ckpt -O v21.ckpt

# SDZ_Extensions
  if SDZ_Extensions:
    os.chdir(SDZ_webui_path)
    %cd models/VAE
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/VAE/vae-ft-mse-840000-ema-pruned.ckpt
    %cd ..

    if not os.path.exists('Lora'):
        os.makedirs('Lora')
    %cd Lora
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/epiNoiseoffset_v2.safetensors
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/theovercomer8sContrastFix_sd15.safetensors
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/Lora/to8sHighKeyLORASD15SD2_sd15.safetensors
    %cd ../..

    %cd embeddings
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/embeddings/bad-hands-5.pt
    !wget --timestamping https://huggingface.co/uwg/modelz_base/resolve/main/embeddings/easynegative.safetensors
    %cd ..

    %cd extensions
    !git clone https://github.com/Extraltodeus/depthmap2mask.git
    !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git
    !git clone https://github.com/camenduru/sd-civitai-browser.git
    !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
    !git clone https://github.com/fkunn1326/openpose-editor.git
    !git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git
    !git clone https://github.com/opparco/stable-diffusion-webui-two-shot.git
    !git clone https://github.com/opparco/stable-diffusion-webui-composable-lora.git
    !git clone https://github.com/jexom/sd-webui-depth-lib.git
    !git clone https://github.com/alemelis/sd-webui-ar.git
    !git clone https://github.com/hnmr293/posex.git
    !git clone https://github.com/butaixianran/Stable-Diffusion-Webui-Civitai-Helper.git
    !sed -i 's/dl_url = gr.Textbox(label="Download Url", interactive=False, value=None)/dl_url = gr.Textbox(label="Download Url", value=None, interactive=True, lines=1)/' $SDZ_install'/stable-diffusion-webui/extensions/sd-civitai-browser/scripts/civitai-api.py'
    %cd ..

# SDZ_ControlNet
  if SDZ_ControlNet:
    os.chdir(SDZ_webui_path)
    %cd extensions
    !git clone https://github.com/Mikubill/sd-webui-controlnet.git
    %cd sd-webui-controlnet/models
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_canny-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_color-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_depth-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_openpose-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors
    !wget --timestamping https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_style-fp16.safetensors
    !cp cldm_v15.yaml cldm_v21.yaml {os.path.join(SDZ_webui_path, 'models')}
    %cd ../../..

# SDZ_Upscalers
  if SDZ_Upscalers:
    os.chdir(SDZ_webui_path+'/'+'models')

    if not os.path.exists('ESRGAN'):
        os.makedirs('ESRGAN')
    with open(SDZ_webui_path+'/'+'Upscalers.txt') as f:
        for i, line in enumerate(f):
            if 1 <= i <= 34:
                link = line.strip()
                os.system('wget --timestamping -P ESRGAN {}'.format(link))

    if not os.path.exists('GFPGAN'):
        os.makedirs('GFPGAN')
    with open(SDZ_webui_path+'/'+'Upscalers.txt') as f:
        for i, line in enumerate(f):
            if 36 <= i <= 38:
                link = line.strip()
                os.system('wget --timestamping -P GFPGAN {}'.format(link))

    if not os.path.exists('SwinIR'):
        os.makedirs('SwinIR')
    with open(SDZ_webui_path+'/'+'Upscalers.txt') as f:
        for i, line in enumerate(f):
            if 40 <= i <= 54:
                link = line.strip()
                os.system('wget --timestamping -P SwinIR {}'.format(link))

# SDZ_Training
  if SDZ_Training == "New_Dreambooth - Kohya":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    %cd extensions
    !git clone https://github.com/ddpn08/kohya-sd-scripts-webui.git
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd ..
    !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 -f https://download.pytorch.org/whl/cu117/torch_stable.html

  elif SDZ_Training == "Old_Dreambooth":
    os.chdir(SDZ_webui_path)
    if not os.path.exists('/usr/local/lib/python3.10/site-packages/open_clip'):
      !wget -N https://github.com/korakot/kora/releases/download/v0.10/py310.sh
      !bash ./py310.sh -b -f -p /usr/local
      !python -m ipykernel install --name "py310" --user
      !pip install open-clip-torch
      !pip install triton
      !sed -i 's@def load_state_dict(checkpoint_path: str, map_location.*@def load_state_dict(checkpoint_path: str, map_location="cuda"):@' /usr/local/lib/python3.10/site-packages/open_clip/factory.py
    
    os.chdir(SDZ_install)
    %cd stable-diffusion-webui
    !git checkout 1130d5df669911a5c67696be90bccca3ecf5f487
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/config.json
    !wget --timestamping https://raw.githubusercontent.com/wilzamguerrero/SDZ/main/SDZ_custom/user_old.css -O user.css
    %cd extensions
    !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git
    %cd sd_dreambooth_extension
    !git checkout f9900ccb3ce7025d1c89334e320a1c3b255bbd07
    %cd ../..

  else:
    SDZ_Training == ""
    os.chdir(SDZ_webui_path)

  if SDZ_Big_Models:
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" $SDZ_webui_path'/modules/sd_models.py'
    !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' $SDZ_webui_path'/modules/shared.py'
    !sed -i "s@map_location='cpu'@map_location='cuda'@" $SDZ_webui_path'/modules/extras.py'
    !sed -i '902s@.*@        self.logvar = self.logvar.to(self.device)@' $SDZ_webui_path'/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py'
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' $SDZ_webui_path'/webui.py'
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda' if os.path.getsize(checkpoint_file) > 5500000000 else map_location@" $SDZ_webui_path'/modules/sd_models.py'
    !sed -i "s@'cpu'@'cuda'@" $SDZ_webui_path'/modules/extras.py'
  else:
    #!sed -i 's@cmd_opts.lowram else \"cuda\"@cmd_opts.lowram else \"cpu\"@' $SDZ_webui_path'/modules/shared.py'
    #!sed -i "s@map_location='cuda'@map_location='cpu'@" $SDZ_webui_path'/modules/extras.py'

    clear_output()
print('[1;90mPreparing SDZ System ✓')

if SDZ_Version == "V1.5":
    !nohup lt -p 7860 > lt.log 2>&1 &
    time.sleep(3)
    webui_link = getLink(SDZ_webui_path+'/'+'lt.log')
    print(webui_link)
    !python launch.py $share

elif SDZ_Version == "V2.1":
    !nohup lt -p 7860 > lt.log 2>&1 &
    time.sleep(3)
    webui_link = getLink(SDZ_webui_path+'/'+'lt.log')
    print(webui_link)
    !python launch.py $share SDZconfigV2